In [1]:
import pandas as pd

In [2]:
questions = pd.read_csv('Data/questions.csv')
answers = pd.read_csv('Data/answers.csv')
survey_year = pd.read_csv('Data/survey.csv')


In [3]:
answers[answers['SurveyID'] == 2014].tail()



,Unnamed: 0,AnswerText,SurveyID,UserID,QuestionID
32755,32755,-1,2014,1256,103
32756,32756,-1,2014,1257,103
32757,32757,-1,2014,1258,103
32758,32758,-1,2014,1259,103
32759,32759,-1,2014,1260,103


1) Look in column to make sure answers are standardized 
    Check and see if you need either remove or change a record to make sure they are all the same 
2) Keep only questions we need for our analysis 
3) Merge it with answer csv
        Try to get questions as columns and answer as rows for each survey taker
4) check for duplicates and missing values 
5) Check column names. 
        Captialize words 
6) 
        

In [4]:
# Merge Questions and answers


survey_data = pd.merge(questions , answers , left_on='questionid' , right_on='QuestionID')

# filter it for questions we need to keep. id 1 , 2 ,3 , 4 ,5 ,14 , 15 , 16 , 32
    # create list with question ids to keep
question_ids = [1 , 2 , 3 , 4 , 5  , 14 , 15 , 16 ] 

filtered_df = survey_data[survey_data['QuestionID'].isin(question_ids)]

#Change column names

filtered_df = filtered_df.rename(columns={'questiontext' : 'Questions' , 'AnswerText' : 'Answers' , 'SurveyID' : 'Year'})

filtered_df

,Unnamed: 0_x,Questions,questionid,Unnamed: 0_y,Answers,Year,UserID,QuestionID
0,0,What is your age?,1,0,37,2014,1,1
1,0,What is your age?,1,1,44,2014,2,1
2,0,What is your age?,1,2,32,2014,3,1
3,0,What is your age?,1,3,31,2014,4,1
4,0,What is your age?,1,4,31,2014,5,1
...,...,...,...,...,...,...,...,...
62443,15,Does your employer offer resources to learn mo...,16,209216,I don't know,2019,4214,16
62444,15,Does your employer offer resources to learn mo...,16,209217,-1,2019,4215,16
62445,15,Does your employer offer resources to learn mo...,16,209218,-1,2019,4216,16
62446,15,Does your employer offer resources to learn mo...,16,209219,No,2019,4217,16


In [5]:
 # Pivoted dataframe to get questions as columns 

pivoted_df = filtered_df.pivot(index='UserID' , columns='Questions', values='Answers' )

# reorder columns

pivoted_df = pivoted_df[['What is your age?' ,'What is your gender?', 'What country do you live in?' , 'If you live in the United States, which state or territory do you live in?' ,
                        'Are you self-employed?' ,'Do you know the options for mental health care available under your employer-provided health coverage?' , 
                          'Does your employer offer resources to learn more about mental health disorders and options for seeking help?',
                        'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
                       ]]

# rename columns 

pivoted_02_df = pivoted_df.rename(columns={'What is your age?' : 'Age' , 'What is your gender?': 'Gender' , 'What country do you live in?':'Country' ,
                                              'If you live in the United States, which state or territory do you live in?': 'State/Territory',
                                              'Are you self-employed?' : 'self-employed?'})


pivoted_02_df.head()




Questions,Age,Gender,Country,State/Territory,self-employed?,Do you know the options for mental health care available under your employer-provided health coverage?,Does your employer offer resources to learn more about mental health disorders and options for seeking help?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?"
UserID,,,,,,,,
1,37,Female,United States,Illinois,-1,NaN,NaN,NaN
2,44,Male,United States,Indiana,-1,NaN,NaN,NaN
3,32,Male,Canada,-1,-1,NaN,NaN,NaN
4,31,Male,United Kingdom,-1,-1,NaN,NaN,NaN
5,31,Male,United States,Texas,-1,NaN,NaN,NaN


In [6]:
# Update male and female to capital letters and group other genders into 'other'

for index , row in pivoted_02_df.iterrows():
      if row[1] == 'male' or row[1] == 'Male':
            pivoted_02_df['Gender'] = pivoted_02_df['Gender'].replace(row[1] , 'Male')
      elif row[1] == 'female' or row[1] == 'Female':
            pivoted_02_df['Gender'] = pivoted_02_df['Gender'].replace(row[1] , 'Female')
      else:
            pivoted_02_df['Gender'] = pivoted_02_df['Gender'].replace(row[1] , 'Other')
    


pivoted_02_df['Gender'].value_counts()

Male      3042
Female    1024
Other      152
Name: Gender, dtype: int64

In [7]:
# Change United States of America to United States. Look at other values

pivoted_02_df['Country'] = pivoted_02_df['Country'].replace('United States of America' , 'United States')


pivoted_02_df['Country'].value_counts(dropna=False)


United States     2604
United Kingdom     482
Canada             199
Germany            136
Netherlands         98
                  ... 
Slovenia             1
Macedonia            1
Venezuela            1
Ghana                1
Mauritius            1
Name: Country, Length: 80, dtype: int64

In [8]:
# Look at -1s. Could be either a diferent country where field is not applicable or an american who 
# did not fill it in 
# There is 12 instances where the country is United States and state/territory is not filled in. Will makes these 'Not Provided'
# Will mark 'Not Applicable' for other countries
            #pivoted_02_df[(pivoted_02_df['State/Territory'] == '-1') & (pivoted_02_df['Country'] == 'United States')]

for index , row in pivoted_02_df.iterrows():
    if row[2] == 'United States' and row[3] == '-1':
        pivoted_02_df.at[index, 'State/Territory'] = 'Not Provided'
    elif row[2] != 'United States' and row[3] == '-1':
        pivoted_02_df.at[index, 'State/Territory'] = 'Not Applicable'




pivoted_02_df['State/Territory'].value_counts(dropna=False)

Not Applicable          1610
California               382
Illinois                 223
Washington               168
New York                 146
Tennessee                121
Texas                    119
Ohio                     109
Michigan                 108
Minnesota                101
Oregon                    99
Indiana                   96
Pennsylvania              94
Massachusetts             76
Nebraska                  61
Colorado                  60
Florida                   52
North Carolina            51
Georgia                   49
Virginia                  46
Wisconsin                 43
Maryland                  42
Missouri                  42
Utah                      29
Kansas                    27
New Jersey                27
Oklahoma                  20
Alabama                   19
Arizona                   19
Iowa                      18
New Hampshire             15
Kentucky                  14
Connecticut               13
Not Provided              12
South Carolina

In [25]:
# Probably safe to say that the majority of people will not be self employed. So, 0 is probably No and 1 
# is probably Yes. -1 must be NaN value
# Dropping self-employed column as I do not need for my analysis. Just needed it to investigate 0 , 1 , -1 values

pivoted_02_df['self-employed?'].value_counts(dropna=False)



0     3550
1      650
-1      18
Name: self-employed?, dtype: int64

In [15]:
# Change -1 and NaN to 'Did not answer'. I do not want to drop them outright as it is telling that they did not answer the question. I also change 'I am not sure' to 'No'

for index , row in pivoted_02_df.iterrows():
    if row[5] == '-1':
        pivoted_02_df.at[index, 'Do you know the options for mental health care available under your employer-provided health coverage?'] = 'Did Not Answer'
    if row[5] == 'I am not sure':
         pivoted_02_df.at[index, 'Do you know the options for mental health care available under your employer-provided health coverage?'] = 'No'

pivoted_02_df['Do you know the options for mental health care available under your employer-provided health coverage?'].fillna(value='Did Not Answer' , inplace=True)


pivoted_02_df['Do you know the options for mental health care available under your employer-provided health coverage?'].value_counts(dropna=False)


Did Not Answer    2027
No                1322
Yes                869
Name: Do you know the options for mental health care available under your employer-provided health coverage?, dtype: int64

In [16]:
pivoted_02_df['Does your employer offer resources to learn more about mental health disorders and options for seeking help?'].value_counts(dropna=False)

for index , row in pivoted_02_df.iterrows():
    if row[6] == '-1':
        pivoted_02_df.at[index, 'Does your employer offer resources to learn more about mental health disorders and options for seeking help?'] = 'Did Not Answer'

pivoted_02_df['Does your employer offer resources to learn more about mental health disorders and options for seeking help?'].fillna(value='Did Not Answer' , inplace=True)


pivoted_02_df['Does your employer offer resources to learn more about mental health disorders and options for seeking help?'].value_counts(dropna=False)

Did Not Answer    1764
No                1084
Yes                702
I don't know       668
Name: Does your employer offer resources to learn more about mental health disorders and options for seeking help?, dtype: int64

In [17]:
for index , row in pivoted_02_df.iterrows():
    if row[7] == '-1':
        pivoted_02_df.at[index, 'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?'] = 'Did Not Answer'

pivoted_02_df['Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?'].fillna(value='Did Not Answer' , inplace=True)



pivoted_02_df['Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?'].value_counts(dropna=False)

Did Not Answer    1764
No                1626
Yes                599
I don't know       229
Name: Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?, dtype: int64

In [28]:
# Dropping self-employed column as I do not need for my analysis. Just needed it to investigate 0 , 1 , -1 values

pivoted_02_df.drop('self-employed?' , axis=1 , inplace=True)



In [29]:
pivoted_02_df.to_csv("Data/Survey_Data.csv" , header=True)